# Logging in python

Logging is a handy tool in a programmer’s toolbox. 

Logs provide developers with an ability to monitor in detail the progress and errors and flow of information in an application. 

If an error occurs, logs can provide more insights than a stack trace by telling you what the state of the program was before it arrived at the line of code where the error occurred.

By logging useful data from the right places, you can find and fix errors easily but also use the data to analyze the performance of the application to plan for scaling. 

## The logging funnel

Logging encourages you to log a lot of data, and then gives multiple mechanisms for filtering out that data. It’s similar to a funnel: lots of data goes in, but most of it gets filtered out before leaving the system as formatted logs.

That leads to simple application code that stays the same (just log everything!) from environment to environment. 

Filters can then be configured via settings and config files to match the environment: if you’re debugging locally you may want to filter nothing out and create log records of everything, but on production, you only want to know about errors, in which case you can add more filters.


## Historical background: Log4J

Logging in python is heavily influenced by a logging paradigm popularized by the __log4j__ java library.

## The Logging Module

The logging module in Python is simply named __`logging`__:

In [ ]:
import logging

With the logging module imported, you can use a “__logger__” to log messages that you want to see.
The logging module provides you with a default logger (the `root` logger) that allows you to get started without needing to do much configuration.

By default, there are 5 standard levels indicating the *severity* of events. Each has a corresponding method that can be used to log events at that level of severity. The defined levels, in order of increasing severity, are the following:

   * DEBUG      
   * INFO     
   * WARNING    
   * ERROR    
   * CRITICAL    
   
The corresponding methods for each level can be called as shown in the following example:

In [1]:
import logging
logger = logging.getLogger('my_logger')

logging.basicConfig()

logger.debug('This is a debug message ')
logger.info('This is a info message ')
logger.warning('This is a warning message ')
logger.error('This is an error message ')
logger.critical('This is a critical message ')

ERROR:my_logger:This is an error message 
CRITICAL:my_logger:This is a critical message 


Here is what happening :

1. The output lines start with `WARNING` / `ERROR` / `CRITICAL` which are the severity levels we used
2. followed by `my_logger:` - this is the name of the __logger__ we are logging through
3. followed by our custom error message

This format, which shows the level, name, and message separated by a colon (:), is the default __output format__ (it can be customized to include things like timestamp, line number, and other details).

__Note__: the `debug()` and `info()` messages didn’t get logged because, by default, the logging module logs the messages with a severity level of `WARNING` or above (you can change that by configuring the logging module if you want). 

## Basic Configurations

You can use the __`basicConfig(**kwargs)`__ method to configure the "*root*" logger.

The default setting in `basicConfig()` is to set the logger to write to the console in the following format:

```
SEVERITY:logger name:error message
```

Some of the commonly used parameters for `basicConfig()` are the following:

- `level`: The logger will be set to the specified severity level.
- `filename`: This specifies a file to log messages.
- `filemode`: If filename is given, the file is opened in this mode. The default is __a__, which means append.
- `format`: This is the format of the log message.

__Example__: By using the `level` parameter, you can set what level of log messages you want to record, and this would enable all logging calls at or above that level to be logged


In [5]:
from importlib import reload
import logging
reload(logging)

logger = logging.getLogger('my_logger')

logging.basicConfig(
    level=logging.DEBUG # All events at or above DEBUG level will now get logged.
    
    )
# This function does nothing if the root logger already has handlers configured for it.
# jupyter calls basicConfig and set handler automatically hence the use of reload()

logger.debug('This WILL get logged')

<module 'logging' from 'C:\\Anaconda3\\lib\\logging\\__init__.py'>

DEBUG:my_logger:This WILL get logged


It should be noted that usually calling `basicConfig()` to configure the root logger works only if the root logger has not been configured before: this function should usually only be called __once__ hence the use of `reload()` in the above example.

## Logging to file

`basicConfig()`can be used to configure logging to output to a file rather than the console. 

`filename` and `filemode` parameters are used, and you can decide the format of the message using `format`. 

In [16]:
from importlib import reload
import logging
reload(logging)

logger = logging.getLogger('my_logger')

logging.basicConfig(
    filename='app.log', # write to this file
    filemode='a', # open in append mode (the default)
    format='%(name)s - %(levelname)s - %(message)s'
    )
# logging does not write to console anymore, but instead to the specified file:
logging.warning('This will get logged to a file')

<module 'logging' from 'C:\\Anaconda3\\lib\\logging\\__init__.py'>

In [17]:
from pathlib import Path
print(Path('app.log').read_text())

root - WARNING - This will get logged to a file



## Formatting the Output

While you can pass any variable that can be represented as a string from your program as a message to your logs, there are some basic elements that are already a part of the `LogRecord` and can be easily added to the output format. If you want to log the process ID along with the level and message, you can do something like this:

In [18]:
from importlib import reload
import logging
reload(logging)

logger = logging.getLogger('my_logger')

logging.basicConfig(format='ID:%(process)d - %(levelname)s - %(message)s')
logger.warning('This Warning contains the process ID of the process who logged it')

<module 'logging' from 'C:\\Anaconda3\\lib\\logging\\__init__.py'>

ID:34808 - WARNING - This Warning contains the process ID of the process who logged it


`format=` can take a string with `LogRecord` attributes in any arrangement you like.

In [6]:
from importlib import reload
import logging
reload(logging)

logger = logging.getLogger('my_logger')
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logger.info('This message has a date/time timestamp')

<module 'logging' from 'C:\\Anaconda3\\lib\\logging\\__init__.py'>

2021-11-02 17:57:34,645 - This message has a date/time timestamp


<class 'logging.Logger'>


`%(asctime)s` adds the time of creation of the `LogRecord`. The format can be changed using the `datefmt` attribute, which uses the same formatting language as the formatting functions in the `datetime` module, such as `time.strftime()`:

In [20]:
from importlib import reload
import logging
reload(logging)

logger = logging.getLogger('my_logger')
logging.basicConfig(format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logger.warning('Date looks different for this message')


<module 'logging' from 'C:\\Anaconda3\\lib\\logging\\__init__.py'>

27-Oct-21 21:18:30 - Date looks different for this message


## Classes and Functions of the logging module

A default logger, named __`root`__, is used by the __`logging`__ module whenever its functions are called directly like this: `logging.debug()`. 

You can define your own logger by creating an object of the __`Logger`__ class (this is recommended if your application is composed of multiple modules). 

The most commonly used classes defined in the __`logging`__ module are the following:

- `Logger`: This is the class whose objects will be used in the application code directly to call the functions.

- `LogRecord`: Loggers automatically create LogRecord objects that have all the information related to the event being logged: the name of the logger, the function, the line number, the message, and more.

- `Handler`: Handlers send the LogRecord to the required output destination, like the console or a file. Handler is a base for subclasses like StreamHandler, FileHandler, SMTPHandler, HTTPHandler, and more. These subclasses send the logging outputs to corresponding destinations, like sys.stdout or a disk file.

- `Formatter`: This is where you specify the format of the output by specifying a string format that lists out the attributes that the output should contain.




## Using Handlers

Handlers send the log messages to configured destinations like the standard output stream or a file or over HTTP or to your email via SMTP.

A logger that you create can have more than one handler, which means you can set it up to be saved to a log file and also send it over email.

Like loggers, you can also set the severity level in handlers. This is useful if you want to set multiple handlers for the same logger but want different severity levels for each of them. For example, you may want logs with level WARNING and above to be logged to the console, but everything with level ERROR and above should also be saved to a file. 

__Note__: as we will see later in this lesson, people usually use __configuration files__ rather than code, to setup loggers/handlers.

In [23]:
import logging

# Create a custom logger
logger = logging.getLogger(__name__)
logger.propagate = False # do not pass logs to the default logger

# Create handlers
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('file.log', mode='w')
c_handler.setLevel(logging.WARNING)
f_handler.setLevel(logging.ERROR)

# Create formatters and add it to handlers
c_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)

logger.warning('This is a warning')
logger.error('This is an error')

print(open('file.log').read())

__main__ - WARNING - This is a warning
__main__ - WARNING - This is a warning
__main__ - ERROR - This is an error
__main__ - ERROR - This is an error


2021-10-28 20:11:27,390 - __main__ - ERROR - This is an error
2021-10-28 20:11:27,390 - __main__ - ERROR - This is an error



Here, `logger.warning()` is creating a `LogRecord` that holds all the information of the event and passing it to all the Handlers that it has: `c_handler` and `f_handler`.

- `c_handler` is a `StreamHandler` with level `WARNING` and takes the info from the LogRecord to generate an output in the format specified and prints it to the console. 
- `f_handler` is a `FileHandler` with level `ERROR`, and it ignores this LogRecord as its level is WARNING.

When `logger.error()` is called, `c_handler` behaves exactly as before, and `f_handler` gets a LogRecord at the level of `ERROR`, so it proceeds to generate an output just like c_handler, but instead of printing it to console, it writes it to the specified file

## Using configuration files

While it is possible to configure logging as shown above using the module and class functions, it is more flexible and useful to use configuration files for this

We can configure logging by creating a config file or a dictionary and loading it using `fileConfig()` or `dictConfig()` respectively. These are useful in case you want to change your logging configuration in a running application.

Here’s an example file configuration:

The same configuration can be written in a YAML format:

```

formatters:
  simple:
    format: '%(asctime)s | %(levelname)-7s | %(name)s - %(message)s'
handlers:
  console:
    class: logging.StreamHandler
```
In the above file, there are two loggers, one handler, and one formatter. After their names are defined, they are configured by adding the words logger, handler, and formatter before their names separated by an underscore.

To load this config file, you have to use `fileConfig()`.

In [ ]:
import logging
import logging.config

logging.config.fileConfig(fname='example.conf', disable_existing_loggers=False)

# Get the logger specified in the file
logger = logging.getLogger(__name__)

logger.debug('This is a debug message')

## Loggers

Loggers are the application-level interface to the logging system. A system may have multiple different loggers (usually one per module), each with their own name and rules.

We mostly deal directly with the objects of the __`Logger`__ class, which are instantiated using the module-level function `logging.getLogger(name)`. 

__Note__: Multiple calls to `getLogger()` with the same name will return a reference to the same __`Logger`__ object, which saves us from passing the logger objects to every part where it’s needed. 

__Note__: Using `__name__` as the name of the logger helps localize a log message to a particular file in our project: the name of the logger itself would tell us from where the events are being logged.


In [ ]:
logger = logging.getLogger('example_logger')
logger.warning('This is a warning')

### Logger hierarchy

Loggers have a hierarchy: you can create individual loggers and each logger has a parent. 

At the top of the hierarchy is the `__root__` logger (all loggers are descendants of the `root` logger). 

The previous example creates a custom logger named _example_logger_, it inherits implicitly the handler and formatting from the _root_ logger, this is why it outputs to console, and uses the same formatting as the root logger.

A logger hierarchy can be created by asking a parent logger for a new child logger:

In [24]:
log_myapp = logging.getLogger("myapp")
log_myapp_ui = log_myapp.getChild("ui")
print(log_myapp_ui.name)        # 'myapp.ui'
print(log_myapp_ui.parent.name) # 'myapp'

myapp.ui
myapp


Or you can (and should!) use dot notation:

In [25]:
log_myapp_ui = logging.getLogger("myapp.ui")
print(log_myapp_ui.name)        # 'myapp.ui'
print(log_myapp_ui.parent.name) # 'myapp'

myapp.ui
myapp


__Note__: the logger names don’t include the `root` logger. 

The function call `log=logging.getLogger('debug0.x')` creates a logger `x` which is a child of `debug0` which itself is a child of the `root` logger. 

Each logger passes log messages on to its parent: when logging to the logger `log`, it will pass on the message to its parent, and its parent will pass the message to the root logger. 

__Note__: when you use `basicConfig()` what it does is configure the root logger.

## Filters

To create a custom `Filter` you must implement a subclass of `logging.Filter`.

It will have one method, `filter(record)`, that examines the log record and returns `True` to log it or `False` to discard it. 

Then you can install the filter on either a `Logger` or a `Handler` by calling its `addFilter(filter)` method.


In [7]:
class NoParsingFilter(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith('parsing')
import logging

# Create a custom logger
logger = logging.getLogger(__name__)
logger.addFilter(NoParsingFilter())
logger.warning("parsing file1 problem")
logger.warning("another problem")
logger.warning("parsing file2 problem")

2021-11-02 18:23:55,126 - another problem


## Exceptional logging of exceptions

You can use the __`logging.exception()`__ from within the except: handler/block to log the current exception along with the trace information, prepended with a message.

Note that using `logging.exception()`, the severity level is ERROR.

This method should only be called from an exception handler.
```
try:
    # code in here
except Exception as e:
    logging.exception(e)
```

__Note__: Exception info is added to the logging message. 

The use of the `exc_info` argument is an alternative `logging.exception()`, that gives the possibility to use any severity level:
```
try:
    # code in here
except Exception as e:
    logging.error(e, exc_info=True)
```

In [27]:
# Get the logger specified in the file
logger = logging.getLogger(__name__)

logger.debug('This is a debug message')

try:
    print(10/0)
except:
    logging.exception("Problem ...")

28-Oct-21 21:35:55 - Problem ...
Traceback (most recent call last):
  File "<ipython-input-27-15addf783904>", line 7, in <module>
    print(10/0)
ZeroDivisionError: division by zero
